# Temporal Plot Over 24 Hour Cycles
Here we look at cycles of 24 hours of the moments

We calculate mean and such over 24 hours, which in a plot will produce a 3d curve over 24 days, so 24 points to the curve

End result: 24 points for each mice = 1 curve for each mice
16 curves for 16 mice each for mean, variance, skewness, kurtosis

The control mice are HEALTHY: 7,8,15,16



In [2]:
import pandas as pd
from scipy import stats
import numpy as np

mouse_df_list = list()

# pull 16 mice PCA data
for i in range(1,17):
    df = pd.read_excel("data/newmicedata/PCFAnimal{}.xlsx".format(i))
    mouse_df_list.append(df)

for i in range(len(mouse_df_list)):
    mouse_df_list[i] = mouse_df_list[i].drop(['Unnamed: 0'], axis=1)


mouse_df_list[0]

,DateTime,principal component 1,principal component 2,principal component 3
0,2022/11/04 14:01:00,17.132797,1.502005,-6.294937
1,2022/11/04 14:31:00,16.811791,2.291058,-7.172389
2,2022/11/04 15:01:00,15.028200,2.660764,-6.699671
3,2022/11/04 15:31:00,14.183293,2.758718,-5.704350
4,2022/11/04 16:01:00,11.949887,3.195511,-4.635048
...,...,...,...,...
1136,2022/11/28 06:01:00,-1.396782,-0.328081,-1.957727
1137,2022/11/28 06:31:00,-4.110406,0.404820,-0.490767
1138,2022/11/28 07:01:00,-0.687362,-0.372090,-2.771991
1139,2022/11/28 07:31:00,1.670424,-0.888519,-3.420816


In [3]:
# basically take moments every 48 rows

# Example 3: Get the first n rows of specified columns 
# print(df[['Courses', 'Fee', 'Duration', 'Discount']].iloc[:n])
print(mouse_df_list[0][['principal component 1']].iloc[:24])
n = 1141/48
n

    principal component 1
0               17.132797
1               16.811791
2               15.028200
3               14.183293
4               11.949887
5               12.119507
6               12.755418
7                2.407277
8                1.351935
9               13.223974
10              15.406819
11              10.910116
12              10.427273
13              14.212273
14              13.928744
15              12.099684
16               2.003236
17              -2.649612
18              -3.015082
19               3.434211
20              -0.824567
21              -1.574445
22              -3.373616
23              -2.635874


23.770833333333332

In [23]:
# 23 different points (24th is not a complete 24 hours)
DATAPOINTS = 23
NUMCOMPONENTS = 3

# these arrays contain arrays that represent curves, which contain 23 "points" each (arrays) which contain the 3 x y z coordinates
means = []
covs = []
skews = []
kurts = []
for mouse in mouse_df_list:
    # mouse_curve has 23 points in it
    mouse_curve_mean = []
    mouse_curve_cov = []
    mouse_curve_skew = []
    mouse_curve_kurt = []
    for i in range(1, DATAPOINTS + 1):
        mouse_mean = []
        mouse_cov = []
        mouse_skew = []
        mouse_kurt = []
        for j in range(1, NUMCOMPONENTS + 1):
            # grab the first 24 rows of that column
            # mouse is mouse n's dataFrame
            column = (mouse[['principal component {}'.format(j)]].iloc[(i * 24) - 24:i * 24]).to_numpy()
            # column is an array of arrays with one number in each inner array so refactor to just one array with 24 elements
            column_refactor = []
            for n in range(len(column)):
                column_refactor.append(column[n][0])
            # run moment calculations
            # column = df.to_numpy()
            col_moments = stats.describe(column_refactor)
            mouse_mean.append((col_moments).mean)
            mouse_cov.append((col_moments).variance)
            mouse_skew.append((col_moments).skewness)
            mouse_kurt.append((col_moments).kurtosis)
        mouse_curve_mean.append(mouse_mean)
        mouse_curve_cov.append(mouse_cov)
        mouse_curve_skew.append(mouse_skew)
        mouse_curve_kurt.append(mouse_kurt)
    means.append(mouse_curve_mean)
    covs.append(mouse_curve_cov)
    skews.append(mouse_curve_skew)
    kurts.append(mouse_curve_kurt)

mouse_curve_kurt



[[1.1414281337730738, 1.0674133961044951, -0.1230841951956263],
 [-1.4148541922474096, -1.3482612742609783, -1.1599833307227292],
 [-1.736665599799521, -1.6051353664315053, -1.6413076041412977],
 [-1.5352835577012958, -1.3671120837502548, -1.5081615452645025],
 [-1.638422732408969, -1.5345002426312258, -0.7944362597944434],
 [-1.4839650848634172, -1.4083244604979361, -0.8577861870953511],
 [-1.519587029991543, -1.3779186027806958, -1.0580643474161016],
 [-1.3443998378818145, -1.3773866896968252, -0.7383033739630771],
 [-1.1858436398595098, -1.3603043332616316, 0.28167335134798677],
 [-1.4410842774731079, -1.2622996962197108, 1.175720136079013],
 [-1.1943093074127042, -1.3050727782149312, 0.733575548655645],
 [-1.3426505214512405, -1.1923721682494253, -0.1734632974702106],
 [-1.2594391351908112, -1.1733034046094646, 0.892739481387864],
 [-0.3428139071152829, -1.0325663637441131, 2.9200869599590975],
 [-1.2563477932109313, -1.2471718745987879, -0.2489737443785094],
 [-1.5194534050890451,

# Plotting the Curves

This part is achieved through taking the arrays and plotting 23 mean points for a certain mouse to create a curve in 3d space